### Weight Distribution Comparison Metrics

In [ ]:
import tensorflow as tf
import torch
import torchvision
import pytorch_lightning as pl
from torch import nn
import statistics
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from math import log2

In [4]:
config = config = {'learning_rate': .001, 'dropout': 0.2, 'batch_size': 64, 'epochs': 25}

In [1]:
def get_tf_weights(config):
    mnist = tf.keras.datasets.mnist

    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(config['dropout']),
      tf.keras.layers.Dense(10, activation='softmax')
    ])

    opt = tf.keras.optimizers.Adam(learning_rate=config['learning_rate'])

    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    res = model.fit(x_train, y_train, epochs=config['epochs'], batch_size=config['batch_size'])
    res_test = model.evaluate(x_test, y_test)
    just_tf_weights = list()
    # get weights
    for w in model.weights:
        just_tf_weights.extend(w.numpy().flatten())
    # scale the weights
    scaled_weights = MinMaxScaler().fit_transform(np.array(just_tf_weights).reshape(-1, 1))+1
    return scaled_weights

In [5]:
class NumberNet(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(), 
            nn.Linear(784, 128), 
            nn.ReLU(), 
            nn.Dropout(config['dropout']), 
            nn.Linear(128, 10)) ### no softmax because it's included in cross entropy loss
        self.criterion = nn.CrossEntropyLoss()
        self.config = config
        self.test_loss = None
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(torchvision.datasets.MNIST("~/resiliency/", train=True, 
                                                                      transform=torchvision.transforms.ToTensor(), target_transform=None, download=True), 
                                           batch_size=int(self.config['batch_size']))
    
    def test_dataloader(self):
        return torch.utils.data.DataLoader(torchvision.datasets.MNIST("~/resiliency/", train=True, 
                                                                      transform=torchvision.transforms.ToTensor(), target_transform=None, download=True), 
                                           batch_size=int(self.config['batch_size']))
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.config['learning_rate'])
        return optimizer
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.criterion(logits, y)
        logs = {'train_loss': loss}
        return {'loss': loss}
    
    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        logits = self.forward(x)
        loss = self.criterion(logits, y)
        logs = {'test_loss': loss}
        return {'test_loss': loss, 'logs': logs}
    
    def test_epoch_end(self, outputs):
        loss = []
        for x in outputs:
            loss.append(float(x['test_loss']))
        avg_loss = statistics.mean(loss)
        tensorboard_logs = {'test_loss': avg_loss}
        self.test_loss = avg_loss
        return {'avg_test_loss': avg_loss, 'log': tensorboard_logs}

In [7]:
def get_pt_weights(config):
    model = NumberNet(config)
    trainer = pl.Trainer(max_epochs=config['epochs'])
    trainer.fit(model)
    trainer.test(model)
    pt_model_weights = list(model.parameters())
    just_pt_weights = list()
    for w in pt_model_weights:
        just_pt_weights.extend(w.detach().numpy().flatten())
    pt_weights_scaled = MinMaxScaler().fit_transform(np.array(just_pt_weights).reshape(-1, 1))+1

In [8]:
all_tf_weights = list()
for i in range(10):
    all_tf_weights.append(get_tf_weights(config))

Epoch 1/25
938/938 [==============================] - 1s 2ms/step - loss: 0.3291 - accuracy: 0.9054
Epoch 2/25
938/938 [==============================] - 1s 1ms/step - loss: 0.1613 - accuracy: 0.9535
Epoch 3/25
938/938 [==============================] - 1s 2ms/step - loss: 0.1202 - accuracy: 0.9651
Epoch 4/25
938/938 [==============================] - 1s 2ms/step - loss: 0.0967 - accuracy: 0.9711
Epoch 5/25
938/938 [==============================] - 1s 2ms/step - loss: 0.0821 - accuracy: 0.9745
Epoch 6/25
938/938 [==============================] - 1s 1ms/step - loss: 0.0715 - accuracy: 0.9777
Epoch 7/25
938/938 [==============================] - 1s 2ms/step - loss: 0.0646 - accuracy: 0.9795
Epoch 8/25
938/938 [==============================] - 1s 1ms/step - loss: 0.0564 - accuracy: 0.9821
Epoch 9/25
938/938 [==============================] - 1s 2ms/step - loss: 0.0517 - accuracy: 0.9840
Epoch 10/25
938/938 [==============================] - 2s 2ms/step - loss: 0.0489 - accuracy: 0.9839

In [15]:
mean_tf_weights = np.mean(all_tf_weights, axis=0)

In [16]:
all_pt_weights = list()
for i in range(10):
    all_pt_weights.append(get_pt_weights(config))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/Users/mzvyagin/miniconda3/envs/resiliency/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | Sequential       | 101 K 
1 | criterion | CrossEntropyLoss | 0     


Epoch 0:   0%|          | 3/938 [00:00<00:28, 32.63it/s, loss=2.252, v_num=3]

/Users/mzvyagin/miniconda3/envs/resiliency/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Epoch 24: 100%|██████████| 938/938 [00:10<00:00, 85.54it/s, loss=0.015, v_num=3]

Saving latest checkpoint..


...

Epoch 24: 100%|██████████| 938/938 [00:10<00:00, 86.69it/s, loss=0.018, v_num=12]

Saving latest checkpoint..


Testing:   2%|▏         | 17/938 [00:00<00:05, 164.02it/s]

/Users/mzvyagin/miniconda3/envs/resiliency/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing:  99%|█████████▉| 930/938 [00:05<00:00, 185.87it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_loss': 0.0066498999786107115, 'test_loss': 0.0066498999786107115}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 938/938 [00:05<00:00, 184.75it/s]


In [17]:
mean_pt_weights = np.mean(all_pt_weights, axis=0)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [ ]:
# calculate the kl divergence
def kl_divergence(p, q):
	return sum(p[i] * log2(p[i]/q[i]) for i in range(len(p)))

In [ ]:
kl_divergence(mean_tf_weights, mean_pt_weights)